In [2]:
# Splits the datasets from label-studio export

In [3]:
import json
import os

path_to_json = "project-1-at-2023-01-23-15-16-4b8bba1d.json"

In [4]:
def extract_data(path_to_json, label):
    """Diese Funktion nimmt als Eingabe eine Liste von JSON-Objekten und den gewünschten Label-String und gibt eine Liste von "data"-Objekten zurück, die den gewünschten Label-String enthalten."""

    json_file = open(path_to_json)
    json_data = json.loads(json_file.read())
    data_list = []
    for item in json_data:
        annotations = item.get("annotations")
        for annotation in annotations:
            result = annotation.get("result")
            for res in result:
                rectanglelabels = res.get("value").get("rectanglelabels")
                if label in rectanglelabels:
                    data_list.append(item.get("data"))
    return data_list


#print(extract_data(json_data, "woman"))
labels_to_train_datasets = ["woman", "man", "child"]

def extract_test_json_for_labels(labels):

    result = []
    for label in labels:
        result = result + extract_test_json_for_label(path_to_json, label , result)
        result = list(set(result))

    with open("test_datasets.json", "w") as f:
        json.dump(result, f)


def extract_test_json_for_label(path_to_json, label , result):
    json_file = open(path_to_json)
    json_data = json.loads(json_file.read())
    counter = 0


    for item in json_data:
        for annotation in item["annotations"]:
            for rect in annotation["result"]:
                if label in rect["value"]["rectanglelabels"] and counter < 100 and item["file_upload"] not in result:
                    result.append(item["file_upload"])
                    counter += 1

    return result

extract_test_json_for_labels(labels_to_train_datasets)

In [5]:
def move_images_and_labels_to_test_dir():
    with open("test_datasets.json", "r") as file:
        json_string = file.read()
        loaded_list = json.loads(json_string)

    os.chdir("../")
    path_to_image_dir = os.getcwd() + "/TensorFlow/workspace/project_museum/images"
    os.chdir("./Scripts")
    for image_name in loaded_list:
        #move images
        src_path = path_to_image_dir+"/exported_datasets/"+image_name
        dst_dir = path_to_image_dir+"/test"
        dst_path = os.path.join(dst_dir, image_name)
        try:
            os.rename(src_path, dst_path)
        except:
            print("File \""+image_name+"\" might me already in directory"+ dst_dir)
        #move xml
        xml_name = image_name.split(".")[0]+".xml"
        src_path = path_to_image_dir+"/exported_datasets/"+xml_name
        dst_dir = path_to_image_dir+"/test"
        dst_path = os.path.join(dst_dir, xml_name)
        try:
            os.rename(src_path, dst_path)
        except:
            print("File \""+xml_name+"\" might me already in directory" + dst_dir)


move_images_and_labels_to_test_dir()

In [6]:
import os
def move_train_dataset_to_train_dir():
    os.chdir("../TensorFlow/workspace/project_museum/images")
    os.system("mv exported_datasets/* train/")
    os.chdir("../../../../Scripts")


move_train_dataset_to_train_dir()